In [8]:
import xarray as xr
import os
import tensorflow as tf
from util import *   # ./util.py
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np

print('TensorFlow version:',tf.__version__)

TensorFlow version: 2.10.0


In [13]:
variables = [
        'lat1',
        'lon1',
        'zenith_angle',
        'azimuth_angle',
        'skt',
        'u10n',
        'v10n',
        'u10',
        'v10',
        'sp',
        'msl',
        'fsr',
        'u100',
        'v100',
        'cvl',
        'cvh',
        'tvl',
        'tvh',
        'swvl1',
        'swvl2',
        'swvl3',
        'swvl4',
        'slt',
        'lai_lv',
        'lai_hv',
        'sdfor',
        'stl1',
        'stl2',
        'stl3',
        'stl4',
        'd2m'
    ]

atms_n20 = xr.load_dataset('.\\data\\atms_n20_surface_contribution.nc')
atms_n20 = atms_n20.where(atms_n20.lsm == 1.0, drop=True)
atms_n20 = atms_n20.where(atms_n20.tcc <= 0.1, drop=True)

atms_npp = xr.load_dataset('.\\data\\atms_npp_surface_contribution.nc')
atms_npp = atms_npp.where(atms_npp.lsm == 1.0, drop=True)
atms_npp = atms_npp.where(atms_npp.tcc <= 0.1, drop=True)

x_train = atms_n20[variables].to_array().values.T
x_test  = atms_npp[variables].to_array().values.T
y_train = atms_n20['surface_contribution'].values
y_test  = atms_npp['surface_contribution'].values

x_train_mean = x_train.mean(axis=0)
x_train_std  = x_train.std(axis=0)
y_train_mean = y_train.mean(axis=0)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean

In [3]:
# find the best variables in a shallow network with one hidden, 30 neurons
best_variables = []

remaining_variables = list(np.arange(x_train.shape[1]))
shallow_losses = {}

while len(best_variables) < x_train.shape[1]:
#while len(best_variables) < 2:    
    
    best_var = -1
    best_loss = 999999999
    
    for var in remaining_variables:
        current_variables = sorted(best_variables + [var])
        print('training with variables', str(current_variables))
        
        x_train_subset = x_train[:,current_variables]
        x_test_subset = x_test[:,current_variables]
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train_subset.shape[1]),
            tf.keras.layers.Dense(30, activation='relu'),
            tf.keras.layers.Dense(22)
        ])

        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath='checkpoints\\shallow_h30' + str(current_variables) + '.h5',
            save_weights_only=True,
            save_best_only=True,
            verbose=0
        )

        loss_fn = tf.keras.losses.MeanSquaredError()

        model.compile(optimizer='adam',
                      loss=loss_fn)

        hist = model.fit(x_train_subset, y_train, 
                  epochs=50,
                  validation_data=(x_test_subset, y_test),
                  callbacks=[model_checkpoint])
        
        current_loss = min(hist.history['val_loss'])
        shallow_losses[str(current_variables)] = current_loss
        
        if current_loss < best_loss:
            best_var = var
            best_loss = current_loss
    
    print('Best variable:', best_var)
    best_variables.append(best_var)
    remaining_variables.remove(best_var)
    

training with variables [0]
Epoch 1/50
242/242 [==============================] - 1s 2ms/step - loss: 498.5859 - val_loss: 807.3549
Epoch 2/50
242/242 [==============================] - 0s 2ms/step - loss: 237.4851 - val_loss: 249.1053
Epoch 3/50
242/242 [==============================] - 0s 2ms/step - loss: 83.6545 - val_loss: 405.6630
Epoch 4/50
242/242 [==============================] - 0s 2ms/step - loss: 57.2595 - val_loss: 491.1577
Epoch 5/50
242/242 [==============================] - 0s 2ms/step - loss: 55.1960 - val_loss: 507.5680
Epoch 6/50
242/242 [==============================] - 0s 2ms/step - loss: 55.0310 - val_loss: 505.3069
Epoch 7/50
242/242 [==============================] - 0s 2ms/step - loss: 54.9520 - val_loss: 501.3328
Epoch 8/50
242/242 [==============================] - 0s 2ms/step - loss: 54.8907 - val_loss: 497.5041
Epoch 9/50
242/242 [==============================] - 0s 2ms/step - loss: 54.8239 - val_loss: 502.5969
Epoch 10/50
242/242 [======================

242/242 [==============================] - 1s 2ms/step - loss: 76.0229 - val_loss: 3033.1013
Epoch 30/50
242/242 [==============================] - 1s 2ms/step - loss: 75.4303 - val_loss: 3035.0078
Epoch 31/50
242/242 [==============================] - 1s 2ms/step - loss: 74.9364 - val_loss: 3031.8840
Epoch 32/50
242/242 [==============================] - 1s 2ms/step - loss: 74.4776 - val_loss: 3044.9436
Epoch 33/50
242/242 [==============================] - 1s 2ms/step - loss: 74.2269 - val_loss: 3077.9153
Epoch 34/50
242/242 [==============================] - 1s 2ms/step - loss: 73.9216 - val_loss: 3046.3704
Epoch 35/50
242/242 [==============================] - 1s 2ms/step - loss: 73.6051 - val_loss: 3077.1638
Epoch 36/50
242/242 [==============================] - 1s 2ms/step - loss: 73.3972 - val_loss: 3066.2690
Epoch 37/50
242/242 [==============================] - 1s 2ms/step - loss: 73.2918 - val_loss: 3081.0398
Epoch 38/50
242/242 [==============================] - 1s 2ms/step 

242/242 [==============================] - 1s 2ms/step - loss: 50.1037 - val_loss: 213.2978
Epoch 35/50
242/242 [==============================] - 1s 2ms/step - loss: 50.0041 - val_loss: 198.8714
Epoch 36/50
242/242 [==============================] - 1s 2ms/step - loss: 49.9111 - val_loss: 201.9119
Epoch 37/50
242/242 [==============================] - 1s 2ms/step - loss: 49.8352 - val_loss: 197.4922
Epoch 38/50
242/242 [==============================] - 1s 2ms/step - loss: 49.7357 - val_loss: 215.4677
Epoch 39/50
242/242 [==============================] - 1s 2ms/step - loss: 49.6566 - val_loss: 205.3156
Epoch 40/50
242/242 [==============================] - 1s 2ms/step - loss: 49.5773 - val_loss: 213.6361
Epoch 41/50
242/242 [==============================] - 1s 2ms/step - loss: 49.5040 - val_loss: 201.4375
Epoch 42/50
242/242 [==============================] - 1s 2ms/step - loss: 49.4178 - val_loss: 199.3563
Epoch 43/50
242/242 [==============================] - 1s 2ms/step - loss: 4

Epoch 12/50
242/242 [==============================] - 1s 2ms/step - loss: 428.9646 - val_loss: 1505.8850
Epoch 13/50
242/242 [==============================] - 1s 2ms/step - loss: 425.1429 - val_loss: 1512.7980
Epoch 14/50
242/242 [==============================] - 1s 2ms/step - loss: 421.8508 - val_loss: 1519.3300
Epoch 15/50
242/242 [==============================] - 1s 2ms/step - loss: 418.7457 - val_loss: 1508.2815
Epoch 16/50
242/242 [==============================] - 1s 2ms/step - loss: 416.1227 - val_loss: 1531.2856
Epoch 17/50
242/242 [==============================] - 1s 2ms/step - loss: 413.7129 - val_loss: 1521.7206
Epoch 18/50
242/242 [==============================] - 1s 2ms/step - loss: 411.6977 - val_loss: 1546.8171
Epoch 19/50
242/242 [==============================] - 1s 2ms/step - loss: 409.9038 - val_loss: 1528.8665
Epoch 20/50
242/242 [==============================] - 1s 2ms/step - loss: 408.3725 - val_loss: 1552.8623
Epoch 21/50
242/242 [=========================

242/242 [==============================] - 0s 2ms/step - loss: 299.0479 - val_loss: 1570.2052
Epoch 17/50
242/242 [==============================] - 0s 2ms/step - loss: 297.2999 - val_loss: 1568.1973
Epoch 18/50
242/242 [==============================] - 0s 2ms/step - loss: 295.8027 - val_loss: 1565.2324
Epoch 19/50
242/242 [==============================] - 1s 2ms/step - loss: 294.2219 - val_loss: 1555.4952
Epoch 20/50
242/242 [==============================] - 1s 2ms/step - loss: 292.9164 - val_loss: 1567.4606
Epoch 21/50
242/242 [==============================] - 1s 2ms/step - loss: 291.6646 - val_loss: 1579.6715
Epoch 22/50
242/242 [==============================] - 0s 2ms/step - loss: 290.3263 - val_loss: 1568.9934
Epoch 23/50
242/242 [==============================] - 1s 2ms/step - loss: 289.2104 - val_loss: 1570.0281
Epoch 24/50
242/242 [==============================] - 1s 2ms/step - loss: 288.1425 - val_loss: 1569.8533
Epoch 25/50
242/242 [==============================] - 1s 

...

Epoch 21/50
242/242 [==============================] - 1s 2ms/step - loss: 12.3963 - val_loss: 176.9751
Epoch 22/50
242/242 [==============================] - 1s 2ms/step - loss: 11.9939 - val_loss: 178.2691
Epoch 23/50
242/242 [==============================] - 1s 2ms/step - loss: 11.7136 - val_loss: 192.4961
Epoch 24/50
242/242 [==============================] - 1s 2ms/step - loss: 11.4156 - val_loss: 184.1480
Epoch 25/50
242/242 [==============================] - 1s 2ms/step - loss: 11.1503 - val_loss: 181.1059
Epoch 26/50
242/242 [==============================] - 1s 2ms/step - loss: 10.9027 - val_loss: 180.4077
Epoch 27/50
242/242 [==============================] - 1s 2ms/step - loss: 10.6737 - val_loss: 176.7113
Epoch 28/50
242/242 [==============================] - 1s 2ms/step - loss: 10.4467 - val_loss: 188.0663
Epoch 29/50
242/242 [==============================] - 1s 2ms/step - loss: 10.2473 - val_loss: 178.8326
Epoch 30/50
242/242 [==============================] - 1s 2ms/st

242/242 [==============================] - 1s 2ms/step - loss: 7.4150 - val_loss: 163.5158
Epoch 50/50
242/242 [==============================] - 1s 2ms/step - loss: 7.3721 - val_loss: 169.7877
Best variable: 16
training with variables [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29]
Epoch 1/50
242/242 [==============================] - 1s 3ms/step - loss: 225.9606 - val_loss: 122.0632
Epoch 2/50
242/242 [==============================] - 1s 2ms/step - loss: 37.6854 - val_loss: 102.7340
Epoch 3/50
242/242 [==============================] - 1s 2ms/step - loss: 27.5874 - val_loss: 106.7192
Epoch 4/50
242/242 [==============================] - 1s 2ms/step - loss: 23.6572 - val_loss: 104.3742
Epoch 5/50
242/242 [==============================] - 1s 2ms/step - loss: 21.1919 - val_loss: 109.5945
Epoch 6/50
242/242 [==============================] - 1s 2ms/step - loss: 19.3136 - val_loss: 114.4676
Epoch 7/50
242/242 [======================

Epoch 2/50
242/242 [==============================] - 1s 2ms/step - loss: 43.6332 - val_loss: 107.1390
Epoch 3/50
242/242 [==============================] - 1s 2ms/step - loss: 35.3997 - val_loss: 104.0373
Epoch 4/50
242/242 [==============================] - 1s 2ms/step - loss: 29.3312 - val_loss: 102.1122
Epoch 5/50
242/242 [==============================] - 1s 2ms/step - loss: 25.6188 - val_loss: 112.3012
Epoch 6/50
242/242 [==============================] - 1s 2ms/step - loss: 23.3009 - val_loss: 114.4452
Epoch 7/50
242/242 [==============================] - 1s 2ms/step - loss: 21.5824 - val_loss: 124.5294
Epoch 8/50
242/242 [==============================] - 1s 2ms/step - loss: 20.2924 - val_loss: 125.0247
Epoch 9/50
242/242 [==============================] - 1s 2ms/step - loss: 19.2388 - val_loss: 123.9116
Epoch 10/50
242/242 [==============================] - 1s 2ms/step - loss: 18.3962 - val_loss: 128.3262
Epoch 11/50
242/242 [==============================] - 1s 2ms/step - los

Epoch 6/50
242/242 [==============================] - 1s 2ms/step - loss: 22.3446 - val_loss: 115.1811
Epoch 7/50
242/242 [==============================] - 1s 2ms/step - loss: 21.0679 - val_loss: 130.7136
Epoch 8/50
242/242 [==============================] - 1s 2ms/step - loss: 19.9633 - val_loss: 124.5552
Epoch 9/50
242/242 [==============================] - 1s 2ms/step - loss: 19.0366 - val_loss: 134.3574
Epoch 10/50
242/242 [==============================] - 1s 2ms/step - loss: 18.0989 - val_loss: 130.7166
Epoch 11/50
242/242 [==============================] - 1s 2ms/step - loss: 17.1990 - val_loss: 140.8284
Epoch 12/50
242/242 [==============================] - 1s 2ms/step - loss: 16.3297 - val_loss: 127.8359
Epoch 13/50
242/242 [==============================] - 1s 2ms/step - loss: 15.5400 - val_loss: 134.9036
Epoch 14/50
242/242 [==============================] - 1s 2ms/step - loss: 14.8169 - val_loss: 132.6374
Epoch 15/50
242/242 [==============================] - 1s 2ms/step -

242/242 [==============================] - 1s 2ms/step - loss: 16.4148 - val_loss: 125.5574
Epoch 10/50
242/242 [==============================] - 1s 2ms/step - loss: 15.5366 - val_loss: 125.6574
Epoch 11/50
242/242 [==============================] - 1s 2ms/step - loss: 14.7835 - val_loss: 130.5327
Epoch 12/50
242/242 [==============================] - 1s 2ms/step - loss: 14.1974 - val_loss: 134.3082
Epoch 13/50
242/242 [==============================] - 1s 2ms/step - loss: 13.6460 - val_loss: 137.5911
Epoch 14/50
242/242 [==============================] - 1s 2ms/step - loss: 13.2062 - val_loss: 137.0333
Epoch 15/50
242/242 [==============================] - 1s 2ms/step - loss: 12.8361 - val_loss: 135.7340
Epoch 16/50
242/242 [==============================] - 1s 2ms/step - loss: 12.5045 - val_loss: 136.0280
Epoch 17/50
242/242 [==============================] - 1s 2ms/step - loss: 12.2098 - val_loss: 138.1455
Epoch 18/50
242/242 [==============================] - 1s 2ms/step - loss: 1

In [4]:
temp = min(shallow_losses.values())

best_inputs = ''

for k,v in shallow_losses.items():
    if v == temp:
        best_inputs = k
        print(k,v)

[4, 10, 15, 27] 74.29512023925781


In [5]:
best_variables

[27,
 4,
 15,
 10,
 14,
 11,
 6,
 13,
 5,
 3,
 2,
 1,
 8,
 23,
 21,
 26,
 22,
 18,
 7,
 0,
 17,
 28,
 19,
 25,
 12,
 29,
 16,
 30,
 9,
 20,
 24]

In [6]:
shallow_losses

{'[0]': 249.1053009033203,
 '[1]': 1369.7056884765625,
 '[2]': 1100.2449951171875,
 '[3]': 482.9549560546875,
 '[4]': 182.8102569580078,
 '[5]': 551.17138671875,
 '[6]': 1405.6527099609375,
 '[7]': 582.3555297851562,
 '[8]': 1334.441650390625,
 '[9]': 1099.4222412109375,
 '[10]': 809.0467529296875,
 '[11]': 622.6468505859375,
 '[12]': 580.5791625976562,
 '[13]': 1322.697998046875,
 '[14]': 418.240478515625,
 '[15]': 1283.9515380859375,
 '[16]': 333.7572021484375,
 '[17]': 1169.9420166015625,
 '[18]': 737.903076171875,
 '[19]': 1176.6162109375,
 '[20]': 1354.1630859375,
 '[21]': 1215.9781494140625,
 '[22]': 283.1687316894531,
 '[23]': 306.203369140625,
 '[24]': 872.330810546875,
 '[25]': 872.4755859375,
 '[26]': 150.21310424804688,
 '[27]': 92.97422790527344,
 '[28]': 108.26116180419922,
 '[29]': 216.89816284179688,
 '[30]': 264.28619384765625,
 '[0, 27]': 99.00566864013672,
 '[1, 27]': 104.21647644042969,
 '[2, 27]': 92.24337005615234,
 '[3, 27]': 96.71875762939453,
 '[4, 27]': 87.9601

In [10]:
import pickle


with open('shallow_losses_new_h30.pickle', 'wb') as pickle_file:
    pickle.dump(shallow_losses, pickle_file)

In [6]:
import pickle

l = {}

with open('shallow_losses_new_h30.pickle', 'rb') as pickle_file:
    l = pickle.load(pickle_file)
    
sorted(l.items(), key= lambda x: x[1])

[('[4, 10, 15, 27]', 74.29512023925781),
 ('[4, 10, 11, 14, 15, 27]', 75.40502166748047),
 ('[4, 10, 14, 15, 27]', 76.5955810546875),
 ('[4, 6, 10, 11, 14, 15, 27]', 78.52603149414062),
 ('[3, 4, 5, 6, 10, 11, 13, 14, 15, 27]', 79.01775360107422),
 ('[1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 18, 21, 22, 23, 26, 27]',
  79.06971740722656),
 ('[4, 10, 14, 15, 27, 28]', 79.30577087402344),
 ('[4, 6, 10, 11, 13, 14, 15, 27]', 79.51296997070312),
 ('[4, 15, 24, 27]', 79.6866226196289),
 ('[2, 3, 4, 5, 6, 10, 11, 13, 14, 15, 27]', 80.13782501220703),
 ('[1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 21, 23, 27]', 80.2890396118164),
 ('[1, 4, 10, 15, 27]', 80.42771911621094),
 ('[4, 15, 27]', 80.66560363769531),
 ('[3, 4, 15, 27]', 80.72335052490234),
 ('[4, 5, 6, 10, 11, 13, 14, 15, 27]', 80.77688598632812),
 ('[1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 27]', 80.80194854736328),
 ('[4, 8, 10, 11, 14, 15, 27]', 80.85549926757812),
 ('[1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 23, 27]', 80.9336624145

In [11]:
current_variables = [4, 10 ,15, 27]

x_train_subset = x_train[:,current_variables]
x_test_subset = x_test[:,current_variables]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(x_train_subset.shape[1]),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(22)
])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)

hist = model.fit(x_train_subset, y_train, 
          epochs=50,
          validation_data=(x_test_subset, y_test))

Epoch 1/50
242/242 [==============================] - 1s 2ms/step - loss: 397.7940 - val_loss: 319.5944
Epoch 2/50
242/242 [==============================] - 0s 2ms/step - loss: 76.0027 - val_loss: 93.4391
Epoch 3/50
242/242 [==============================] - 0s 2ms/step - loss: 56.8569 - val_loss: 85.7016
Epoch 4/50
242/242 [==============================] - 0s 2ms/step - loss: 54.3244 - val_loss: 85.5085
Epoch 5/50
242/242 [==============================] - 0s 2ms/step - loss: 52.8602 - val_loss: 87.0995
Epoch 6/50
242/242 [==============================] - 0s 2ms/step - loss: 51.8116 - val_loss: 91.4883
Epoch 7/50
242/242 [==============================] - 0s 2ms/step - loss: 50.8749 - val_loss: 97.5783
Epoch 8/50
242/242 [==============================] - 0s 2ms/step - loss: 50.0208 - val_loss: 97.9378
Epoch 9/50
242/242 [==============================] - 0s 2ms/step - loss: 49.1890 - val_loss: 101.1071
Epoch 10/50
242/242 [==============================] - 0s 2ms/step - loss: 48.4

In [14]:
atms_npp

<xarray.Dataset>
Dimensions:               (level: 137, obs_id: 1193, channel: 22)
Coordinates:
  * level                 (level) int64 1 2 3 4 5 6 ... 132 133 134 135 136 137
    jd2                   float64 2.458e+06
  * channel               (channel) int64 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22
    lat2                  (obs_id) float64 68.4 68.4 68.4 ... 23.4 23.1 23.1
    lon2                  (obs_id) float64 -166.2 -165.9 ... -109.8 -106.2
Dimensions without coordinates: obs_id
Data variables: (12/151)
    crwc                  (level, obs_id) float64 -5.821e-11 ... -5.821e-11
    cswc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    t                     (level, obs_id) float64 170.4 170.3 ... 302.6 305.4
    q                     (level, obs_id) float64 2.429e-06 ... 0.01498
    o3                    (level, obs_id) float64 1.131e-07 ... 5.596e-08
    clwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                    ...
    scan_line_stddev      (obs_id) float64 0.7071 nan nan ... nan 0.7071 0.7071
    scan_line_count       (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    fov1                  (obs_id) float64 9.0 9.0 10.0 9.0 ... 94.0 86.0 94.0
    fov1_stddev           (obs_id) float64 0.0 nan nan ... nan 4.767e-06 0.0
    fov1_count            (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    surface_contribution  (obs_id, channel) float64 0.6102 0.6098 ... nan nan